In [5]:
from datetime import datetime

# Function to load existing data from the CSV file
def load_existing_data(output_filename):
    if os.path.exists(output_filename):
        df_existing = pd.read_csv(output_filename)
        return df_existing
    else:
        return pd.DataFrame()
    
    # Function to save the aggregated data to a CSV file
def save_all_data_to_csv(aggregated_data, output_filename):
    # Convert the aggregated data to a DataFrame
    df_new = pd.DataFrame.from_dict(aggregated_data, orient='index')
    
    # Ensure that the columns are in the desired order
    fuel_categories = ['BeginDate', 'Coal', 'Hydro', 'Natural Gas', 'Nuclear', 'Oil', 
                       'Other', 'Landfill Gas', 'Refuse', 'Solar', 'Wind', 'Wood']
    
    # Add missing columns with default value 0 if not present
    for fuel in fuel_categories:
        if fuel not in df_new.columns:
            df_new[fuel] = 0
    
    # Reorder the DataFrame to ensure the columns are in the correct order
    df_new = df_new[fuel_categories]

    # Load existing data if the file exists
    df_existing = load_existing_data(output_filename)
    
    if not df_existing.empty:
        # Combine existing and new data
        df_combined = pd.concat([df_existing, df_new])
        # Drop duplicates to ensure no overlap
        df_combined = df_combined.drop_duplicates(subset=['BeginDate'])
    else:
        df_combined = df_new
    
    # Save the combined DataFrame to a CSV file
    df_combined.to_csv(output_filename, index=False)
    print(f"Data saved to {output_filename}")


# Main function to automate queries for a range of dates and store all data in one CSV
def automate_queries_with_auth_and_single_output(start_date, end_date, username, password, output_filename):
    # Load existing data to check the latest date
    df_existing = load_existing_data(output_filename)
    
    # Get the latest date from the existing data (if any)
    if not df_existing.empty:
        df_existing['BeginDate'] = pd.to_datetime(df_existing['BeginDate'])  # Convert to datetime
        latest_date = df_existing['BeginDate'].max().date()  # Extract only the date part
    else:
        latest_date = None  # No existing data, process all dates
    
    current_date = datetime.strptime(start_date, "%Y%m%d").date()
    end_date = datetime.strptime(end_date, "%Y%m%d").date()
    
    # Dictionary to accumulate all data
    aggregated_data = {}
    
    while current_date <= end_date:
        # Only process new dates (after the latest date in the existing data)
        if latest_date is None or current_date > latest_date:
            date_str = current_date.strftime("%Y%m%d")
            
            # Fetch data for the current day with authentication
            data_list = fetch_data_for_day(date_str, username, password)
            
            # Aggregate the data into the main dictionary
            if data_list:
                aggregate_data(data_list, aggregated_data)
        
        # Move to the next day
        current_date += timedelta(days=1)
    
    # Once all data is aggregated, save to the CSV file
    save_all_data_to_csv(aggregated_data, output_filename)

# Example usage
username = 'alean@bu.edu'
password = 'Mq75eg8pxTBCEKY'
start_date = '20221001'
end_date = datetime.today().strftime('%Y%m%d')
output_filename = 'genfuelmix_aggregatedyear.csv'

automate_queries_with_auth_and_single_output(start_date, end_date, username, password, output_filename)

sum


Failed to retrieve data for 20231119. Status code: 429
Failed to retrieve data for 20231120. Status code: 429
Failed to retrieve data for 20231121. Status code: 429
Failed to retrieve data for 20231122. Status code: 429
Failed to retrieve data for 20231123. Status code: 429
Failed to retrieve data for 20231124. Status code: 429
Failed to retrieve data for 20231125. Status code: 429
Failed to retrieve data for 20231126. Status code: 429
Failed to retrieve data for 20231127. Status code: 429
Failed to retrieve data for 20231128. Status code: 429
Failed to retrieve data for 20231129. Status code: 429
Failed to retrieve data for 20231130. Status code: 429
Failed to retrieve data for 20231201. Status code: 429
Failed to retrieve data for 20231202. Status code: 429
Failed to retrieve data for 20231203. Status code: 429
Failed to retrieve data for 20231204. Status code: 429
Failed to retrieve data for 20231205. Status code: 429
Failed to retrieve data for 20231206. Status code: 429
Failed to 